### LOADING DATA

In [3]:
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader(r"C:\Users\LENOVO\Desktop\ML_RAG\RAG\Structured_Data.pdf")
data = loader.load()
print(len(data))


21


### SPLIT THE DATA

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=300
)
chunks = text_splitter.split_documents(data)

print((chunks[0].page_content))
print(len(chunks))

Churn des clients ( télécommunication)
Ce use case a pour objectif de prédire les clients susceptibles de partir afin d’anticiper la
majorité des départs des clients et ainsi le département marketing peut mener des actions
sur cette population pour la fidéliser
* Identifiant de la ligne (numéro de téléphone)
* Ancienneté de la ligne en jours (date courante - date d'activation de la ligne)
* Date d'activation de la ligne
* Date de résiliation de la ligne
* Age du client (en année)
* Catégorie socio-professionnelle
* Ville
* Sexe
* Etat matrimonial
* Type identification (registre du commerce)
* Rapport entre la durée des appels sortants et la durée des appels entrant durant les 6 derniers
mois
* Durée totale des appels (Entrant + Sortant) durant les 6 derniers mois
* Part des appels (Entrant + Sortant) réalisés le week-end parmi la totalité des appels (Entrant +
Sortant)
* Part des appels (Entrant + Sortant) réalisés le soir (20h à 8h) parmi la totalité des appels
(Entrant + Sortant)
* N

### Vector store

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
import os


model_name = "Lajavaness/sentence-camembert-large"  
embeddings_model = HuggingFaceEmbeddings(model_name=model_name)


vector_store = FAISS.from_documents(documents=data, embedding=embeddings_model)
retriever = vector_store.as_retriever()
# return retriever




c:\Users\LENOVO\Desktop\ML_RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Groq 


In [2]:
import os
from langchain_groq import ChatGroq
from langchain_mistralai import ChatMistralAI

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = "your_groq_api_key_here"

groq_llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
groq_llm1 = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


### Hypothetical Document Embedder

In [7]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

# Define the prompt template
def hypothetical_document_generation(query) :
   
    prompt = PromptTemplate.from_template("""Vous etes expert en machine learning, Citer quelques variables necessaires que doit un jeu de donnee contenenir pour resoudre 
    le probleme machine learning de la question suivante : {input}
    ** Decris d'abord le probleme machine learning en deux phrases max .
    Reponse: en français""")

# LLMChain is a chain that takes a prompt and an LLM and runs them together
    llm_chain = LLMChain(llm=groq_llm, prompt=prompt)


# generating the hypothetical document 
    response = llm_chain.run(input=query)
    return response

# Get relevent documents using the hypothetical document instead of the original query to increase the performance of the retriever

# hypothetical_doc = hypothetical_document_generation("Comment predire les transactions frauduleuses")
# print(hypothetical_doc)
# retrieved_docs = retriever.get_relevant_documents(hypothetical_doc, k=1)
# print(retrieved_docs[0].page_content)



In [2]:
%pip list


Package                                  Version
---------------------------------------- -----------
aiohappyeyeballs                         2.6.1
aiohttp                                  3.11.14
aiosignal                                1.3.2
annotated-types                          0.7.0
anyio                                    4.9.0
asgiref                                  3.8.1
asttokens                                3.0.0
attrs                                    25.3.0
backoff                                  2.2.1
bcrypt                                   4.3.0
build                                    1.2.2.post1
cachetools                               5.5.2
certifi                                  2025.1.31
charset-normalizer                       3.4.1
chromadb                                 1.0.10
click                                    8.1.8
colorama                                 0.4.6
coloredlogs                              15.0.1
comm                                 


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Retrieve relevent documets

In [ ]:
def get_relevant_documents(hypothetical_document,original_query):
# Get relevent documents using the hypothetical document instead of the original query to increase the performance of the retriever
    retrieved_docs_1 = retriever.invoke(hypothetical_document,k=3)
    # print(retrieved_docs_1[0].page_content)
# Get relevent documents using the original query
    retrieved_docs_2 = retriever.invoke(original_query,k=3)
    # print(retrieved_docs_2[0].page_content)
# Combine the two sets of documents     
    combined_docs = retrieved_docs_1 + retrieved_docs_2
    # print(len(combined_docs))
# Remove duplicates based on the page content
    unique_docs = {doc.page_content: doc for doc in combined_docs}.values()
    return list(unique_docs) 
    


### Langchain pre-defined function

In [12]:
from langchain.chains import HypotheticalDocumentEmbedder
from langchain_core.prompts import PromptTemplate

# Use a simple prompt for hyde
hyde_prompt = PromptTemplate.from_template(
	"Générez un document hypothétique qui répond à la question suivante : {question}"
)

llm_chain = LLMChain(llm=groq_llm, prompt=hyde_prompt)
hyde = HypotheticalDocumentEmbedder(llm_chain=llm_chain, base_embeddings=embeddings_model)

query = "Comment predire le risque de credit au secteur bancaie ?"
vector = hyde.embed_query(query)

KeyError: 'required'

In [13]:
print(self.llm_chain.input_schema)


NameError: name 'self' is not defined

### Reranking using LLM

In [14]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

prompt = ChatPromptTemplate(
    [
        SystemMessage(content="""Tu es un assistant expert en traitement de texte et en compréhension de documents techniques.

Ta tâche est d’évaluer la **pertinence d’un document** par rapport à une **question en machine learning**.

### Instructions :

1. Lis attentivement la **question utilisateur**.
2. Lis ensuite le **document** fourni.
3. Attribue un **score de pertinence** entre **00** (pas du tout pertinent) et **100** (parfaitement pertinent) selon les critères suivants :
   - Le document répond-il à la question ou aide-t-il à y répondre ?
   - Contient-il des informations directement utiles (données, variables, structure, contexte, logique...) ?
   - Est-ce qu’il permet d’élaborer une solution ou une stratégie machine learning basée sur la question ?
4. Ne repete pas le meme score pour differents documents liee a la meme question, Voir bien la difference et donner un score different pour chaque dpcument
   ** Ranking depuis 90 a 100 est pour le plus pertinent

- Repondez **uniquement** par le score 

---"""),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True,k=4)

legacy_chain = LLMChain(
    llm=groq_llm1,
    prompt=prompt,
    memory=memory,
)



In [ ]:
def clean_retrieved_docs(retrieved_docs, query):
    """
    Clean the retrieved documents based on their relevance to the query.
    """ 
    crd = {}

    for doc in retrieved_docs:
    # Use the LLMChain to get the relevance score
        user_input = f"Document : {doc.page_content} \n Question : {query}"

        # relevance_score = conversation_buf.predict(question=query,document=doc.page_content)
        relevance_score = legacy_chain.invoke({"text": user_input})
        # print(f"Document: {doc.page_content}")
        # print(f"Relevance outcome : {relevance_score}")
        score = int(relevance_score["text"])
        # score = int(relevance_score.content)
        print(score,type(score))
        if score >= 90 :
            crd[doc.page_content] = score
    if crd:
    # Sort the dictionary by value in descending order and take the top 2
        most_relevant = sorted(crd, key=crd.get, reverse=True)[:2]
        return most_relevant
    else :
        return []

In [16]:
def llm_report (query,context) :  
    prompt = PromptTemplate.from_template("""
Tu es un expert en machine learning. Tu vas recevoir :

- Une **question utilisateur** qui décrit une tâche ou un problème à résoudre avec le machine learning.
- Un **contexte** qui contient les informations sur le dataset disponible (features, types, cible, etc.).

Ta tâche est de rédiger un **cahier des charges complet** qui décrit l’approche machine learning à adopter pour répondre à la question, en t’appuyant exclusivement sur le dataset fourni dans le contexte.

Le cahier des charges doit inclure :
- Une reformulation du problème et son objectif.
- Une description du dataset (variables, types, cible, problèmes éventuels).
- Le type de problème (classification, régression...).
- Les étapes de préparation des données.
- Le ou les modèles recommandés, avec justification.
- La stratégie d'entraînement et de validation.
- Les métriques d’évaluation pertinentes.
- Les améliorations possibles (tuning, sélection de variables, etc.).
- Une conclusion synthétique.

Rédige le tout de façon structurée, claire et professionnelle.
Générez le **contenu complet du rapport** en suivant cette structure. Formatez le texte de manière à ce qu’il puisse facilement être converti en **PDF** à l’aide d’un outil markdown-to-pdf ou HTML-to-PDF.

Format d’entrée :

---
Contexte (dataset) :  
{context}

Question utilisateur :  
{question}
---


""")
    reponse = groq_llm.invoke(prompt.format(context=context,question=query))
    return reponse.content

In [ ]:
def final_response (query) :
    hypothetical_doc = hypothetical_document_generation(query)
    relevant_documents = get_relevant_documents(hypothetical_doc,query)
    print("Number of documents before cleaning (just combination) : ",len(relevant_documents))
    clean_documents = clean_retrieved_docs(relevant_documents,query)
    print("Number of documents after LLM as judge scoring, and filter  : ",len(clean_documents))

    context = "\n".join([doc for doc in clean_documents])
    
    if clean_documents :
        response  = llm_report(query,context)
    else:
        response = llm_report(query,hypothetical_doc)
        
    return hypothetical_doc,clean_documents,response
    
    

### Test


In [31]:
input = "Comment predire le churn des client en telecomunications"

hypothetical_document,clean_docs,response = final_response(input)

print(clean_docs[0])

Number of documents before cleaning (just combination) :  3
20 <class 'int'>
98 <class 'int'>
0 <class 'int'>
Number of documents after LLM as judge cleaning  :  1
Churn des clients ( télécommunication)
Ce use case a pour objectif de prédire les clients susceptibles de partir afin d’anticiper la
majorité des départs des clients et ainsi le département marketing peut mener des actions
sur cette population pour la fidéliser
* Identifiant de la ligne (numéro de téléphone)
* Ancienneté de la ligne en jours (date courante - date d'activation de la ligne)
* Date d'activation de la ligne
* Date de résiliation de la ligne
* Age du client (en année)
* Catégorie socio-professionnelle
* Ville
* Sexe
* Etat matrimonial
* Type identification (registre du commerce)
* Rapport entre la durée des appels sortants et la durée des appels entrant durant les 6 derniers
mois
* Durée totale des appels (Entrant + Sortant) durant les 6 derniers mois
* Part des appels (Entrant + Sortant) réalisés le week-end par

In [32]:
print(response)

**Cahier des Charges : Prédiction du Churn des Clients en Télécommunications**

**Reformulation du Problème et Objectif**

Le problème consiste à prédire les clients susceptibles de partir (churn) dans le secteur des télécommunications, afin d'anticiper la majorité des départs des clients et de permettre au département marketing de mener des actions pour fidéliser cette population.

**Description du Dataset**

Le dataset comprend 24 variables, dont 2 variables cibles (Date de résiliation de la ligne et Churn) et 22 variables explicatives. Les variables explicatives sont de type numérique (Ancienneté de la ligne, Age du client, Durée des appels, Nombre de SMS, etc.) et catégorielles (Catégorie socio-professionnelle, Ville, Sexe, Etat matrimonial, etc.). Les variables cibles sont binaires (0 pour les clients qui ne partent pas, 1 pour les clients qui partent).

**Type de Problème**

Le problème est de type classification binaire, où l'objectif est de prédire la probabilité qu'un client p

### QUERY

In [13]:
# -----------QUERY--------------
query = "quelles variables utiliser pour predire le salaire ?"

# retrieved_docs = retriever.invoke(query,k=1)
retrieved_docs = vector_store.similarity_search(query, k=1)

context = "\n".join([doc.page_content for doc in retrieved_docs])
# prompt = f"Contexte:\n{context}\n\nQuestion: {query}\rReponse:"
# ----------------------------

In [14]:
print(context)

Prédiction de Salaire
* Âge : l'âge de chaque employé, en années.
* Genre : le genre de chaque employé, qui peut être soit homme, soit femme.
* Niveau d'études : le niveau d'études de chaque employé, qui peut être : lycée,
licence (bachelor), master ou doctorat (PhD).
* Intitulé du poste : le poste occupé par chaque employé. Les intitulés peuvent
varier selon l'entreprise, par exemple : manager, analyste, ingénieur ou
administrateur.
* Années d'expérience : le nombre d’années d’expérience professionnelle de chaque
employé.
* Salaire : le salaire annuel de chaque employé en dollars américains. Les valeurs
peuvent varier en fonction de facteurs comme l’intitulé du poste, les années
d’expérience et le niveau d’études.


### PROMPT

In [15]:
from langchain_core.prompts import PromptTemplate

template = """
Tu es un assistant expert en machine learning.

Voici un contexte contenant des descriptions de variables extraites d’un dataset :

{context}

Ta tâche est de répondre à la question suivante **en te basant sur les variables présentes dans ce contexte** :

Question : {question}

Réponse :
"""

rag_prompt = PromptTemplate.from_template(template)


In [7]:
from langchain_core.prompts import PromptTemplate

template = """
en te basant uniquement sur ce contexte :

{context}

répondez à la question suivante :

Question : {question}

Réponse :
"""

rag_prompt1 = PromptTemplate.from_template(template)


### LLAMA

In [16]:
from langchain_ollama.llms import OllamaLLM

llm=OllamaLLM(model='llama3.2:latest')
response = llm.invoke(rag_prompt.format(context=context, question=query))
print("Answer:", response)

Answer: Pour prévoir le salaire, il est important de considérer les facteurs qui ont le plus d'influence sur la rémunération. Selon le contexte fourni, voici les variables que je recommanderais de utiliser pour prévoir le salaire :

1. **Âge** : L'âge peut avoir une influence sur la rémunération, car les personnes âgées peuvent avoir plus d'expérience et être considérées comme plus compétentes dans leur domaine.
2. **Niveau d'études** : Le niveau d'études est un facteur clé pour déterminer les compétences et les connaissances acquises par un individu, ce qui peut avoir une influence sur la rémunération.
3. **Intitulé du poste** : L'intitulé du poste est le plus directement lié au salaire, car il reflète la responsabilité et l'importance de l'emploi.
4. **Années d'expérience** : Les années d'expérience sont un facteur clé pour évaluer les compétences et la productivité d'un individu, ce qui peut avoir une influence sur la rémunération.

Il est possible de considérer également d'autres v

In [ ]:
def evaluate_answer(question, context, answer):
    prompt = f"""Vous allez évaluer la réponse d'un assistant virtuel à une question basée sur un contexte donné.

    Question: {question}
    Contextw: {context}
    Reponse: {answer}

    Cette réponse est elle juste pour le context donné ? Répond par :
    - Correcte
    - Incorrecte
    Et expliquez bievement pourquoi .
    
    Si le contexte ne contient pas d'informations pertinentes pour répondre à la question, répondez par "Contexte non correspondant".
    """
    return llm.invoke(prompt)
     


def correct_answer(question, context):
    prompt = f"""La réponse précédente était incorrecte. Veuillez générer une réponse correcte en vous basant strictement sur le contexte.

    Question: {question}
    Context: {context}
    Réponse correcte: 
    """
    return llm.invoke(prompt)

In [ ]:
response = llm.invoke(rag_prompt.format(context=context, question=query))

evaluation = evaluate_answer(query, context, response)


i=0
while "Correcte" not in evaluation:
    i=i+1
    print("Evaluation:", evaluation)
    if "Incorrecte" in evaluation:
        response = correct_answer(query, context)
        # print("Réponse corrigée:", response)
        evaluation = evaluate_answer(query, context, response)
    elif  "Contexte non correspondant" in evaluation:
      query = input("S'il vous plaît, reformulez la question : ")
      retrieved_docs = vector_store.similarity_search(query, k=1)
      context = "\n".join([doc.page_content for doc in retrieved_docs])
      response = llm.invoke(rag_prompt.format(context=context, question=query))
      evaluation = evaluate_answer(query, context, response)
    else:
        print("Erreur d'évaluation, veuillez vérifier le modèle.")
        break

# Sortie de la boucle sigifie que la réponse est correcte
if i>0:
    print(f"Réponse Regénerée {i} fois : ", response)
else:
    print("Réponse initiale correcte:", response)
    




In [149]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

# Step 1: Set up the LLM

# Step 2: Add a memory object
# memory = ConversationSummaryMemory(llm=groq_llm,return_messages=True)

# # Step 3: Create a conversational chain
# conversation = ConversationChain(
#     llm=groq_llm,
#     memory=memory,
#     verbose=True  # shows what's going into the prompt

# )

# Step 4: Interact
# print(conversation.predict(input="Hi, my name is Marouan."))
# print(conversation.predict(input="What is my name?"))
# print(conversation.predict(input="Remember that I love sushi."))
print(conversation.predict(input="what was my name"))




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[SystemMessage(content="Current summary:\nMarouan introduces himself as a data scientist and expresses his love for sushi.\n\nNew lines of conversation:\nAI: Nice to meet you, Marouan. I'm delighted to learn that you're a data scientist. Did you know that the field of data science has its roots in the 1960s, when computer scientists like John Tukey and Peter Naur began exploring the intersection of statistics and computer programming? As for sushi, I'm familiar with the various types of sushi, including maki, nigiri, sashimi, and temaki. Which type of sushi is your favorite, Marouan?\n\nNew summary:\nMarouan introduces himself as a data scientist and expresses his love

In [146]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import ChatMessageHistory


# Prompt moderne
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{text}"),
])

# Mémoire
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

history_store = {}

# Fonction pour retourner une instance de message history par session
def get_message_history(session_id: str):
    if session_id not in history_store:
        history_store[session_id] = ChatMessageHistory()
    return history_store[session_id]
    
# Runnable avec historique
chain = RunnableWithMessageHistory(
    prompt | groq_llm,
    get_session_history=get_message_history,
    input_messages_key="text",
    history_messages_key="chat_history",
)

# Appel de la chaîne (simulateur de session)
# result1 = chain.invoke({"text": "i like football , its my favorite"}, config={"configurable": {"session_id": "user1"}})
# print(result1)

result2 = chain.invoke({"text": "what was my name"}, config={"configurable": {"session_id": "user1"}})
print(result2)


content="I'm a large language model, I don't have the ability to retain information about previous conversations or keep track of individual users. Each time you interact with me, it's a new conversation and I don't have any prior knowledge about you. If you'd like to share your name with me, I'd be happy to chat with you!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 39, 'total_tokens': 109, 'completion_time': 0.093333333, 'prompt_time': 0.002431146, 'queue_time': 0.124832525, 'total_time': 0.095764479}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'stop', 'logprobs': None} id='run-73ddbdd1-e85b-4a1f-bfbc-19a84bcd5734-0' usage_metadata={'input_tokens': 39, 'output_tokens': 70, 'total_tokens': 109}


In [139]:
from typing import Optional

from langchain_community.chat_models import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import BaseChatMessageHistory, InMemoryHistory

store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = prompt | ChatAnthropic(model="claude-2")

chain_with_history = RunnableWithMessageHistory(
    chain,
    # Uses the get_by_session_id function defined in the example
    # above.
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)

print(chain_with_history.invoke(  # noqa: T201
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "foo"}}
))

# Uses the store defined in the example above.
print(store)  # noqa: T201

print(chain_with_history.invoke(  # noqa: T201
    {"ability": "math", "question": "What's its inverse"},
    config={"configurable": {"session_id": "foo"}}
))

print(store)  # noqa: T201

ModuleNotFoundError: No module named 'langchain_core.messages.BaseChatMessageHistory'